In [1]:
Pkg.add("ForwardDiff")
Pkg.add("FactCheck")

INFO: Nothing to be done
INFO: No packages to install, update or remove
INFO: Package database updated


In [1]:
using ForwardDiff, FactCheck

INFO: Recompiling stale cache file /home/lamind-admin/.julia/lib/v0.4/ForwardDiff.ji for module ForwardDiff.
INFO: Recompiling stale cache file /home/lamind-admin/.julia/lib/v0.4/Calculus.ji for module Calculus.


LoadError: LoadError: ArgumentError: FactCheck not found in path
while loading In[1], in expression starting on line 1

In [2]:
using ForwardDiff, Base.Test

# Programação defensiva - Test Driven Development

- Testes a priori

In [6]:
facts("Testes de Brincadeira") do
    @fact 1 --> greater_than(0)
end

Testes de Brincadeira
1 fact verified.


delayed_handler (generic function with 4 methods)

In [8]:
context("Testes parecidos") do
    facts("Testes específicos") do
        @fact factorial(4) --> 24
    end
    
    facts("Outros testes") do
        @fact factorial(3) --> 7
    end
end

  > Testes parecidos
Testes específicos
1 fact verified.
Outros testes
    Failure :: (line:-1) :: Testes parecidos :: fact was false
      Expression: factorial(3) --> 7
        Expected: 7
        Occurred: 6
Out of 1 total fact:
  Failed:   1


delayed_handler (generic function with 4 methods)

In [11]:
@test_approx_eq 1.0 0.99999999999999999

## Exemplo falso

Queremos encontrar uma função que calcula a raiz quadrada de um número $a$.

Objetivo: implementar

    x = raiz(a)

Testes sequenciais:
- Raiz de 0
- Raiz de 1
- Raiz de 4
- Raiz de quadrados perfeitos
- Raiz de 2
- Raiz de negativo
- Raiz de 2 * k^2

Fazer em arquivos:
- teste_raiz.jl
- raiz.jl

In [26]:
?@test_throws

```
@test_throws(extype, ex)
```

Test that the expression `ex` throws an exception of type `extype` and calls the current handler to handle the result.


## TDD em algoritmos matemáticos

e.g. Newton para raiz de funções

Decisão a priori:

    x, ef, fx, fdx, iter, el_time = newton(f, x0; tol_f=1e-6,
        tol_x=1e-3, maxiter=1000, maxtime=60)

Saída:
- x é a solução
- ef é o sinal de saída (exitflag): 0 - funcionou
    outros valores: falhas
- fx = f(x)
- fdx = f'(x)
- iter é o número de iterações
- el_time é o tempo decorrido (elapsed time)

Entrada:
- f: a função
- x0: ponto inicial
- tol\_f: tolerância da derivada: |f(x)| < tol_f
- tol\_x: tolerância do tamanho do passo: |x_{k+1}-x_k| < tol_x
- maxiter: máximo de iterações
- maxtime: máximo de tempo decorrido

In [12]:
context("Exemplos básicos") do
    facts("f(x) = x^2") do
        f(x) = x^2
        x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
        @fact x --> 0.0
        @fact ef --> 0
        @fact fx --> 0.0
        @fact fdx --> 0.0
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
end

  > Exemplos básicos
f(x) = x^2
6 facts verified.


delayed_handler (generic function with 4 methods)

In [32]:
f(x) = x^2
x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
@test_approx_eq x 0.0
@test ef == 0
@test_approx_eq fx 0.0
@test_approx_eq fdx 0.0
@test iter < 1000
@test el_time < 60

f(x) = x^2 + 1.0
x, ef, fx, fdx, iter, el_time = newton(f, 1.0)
@test_approx_eq x 0.0
@test ef == 0
@test_approx_eq fx 1.0
@test_approx_eq fdx 0.0
@test iter < 1000
@test el_time < 60



LoadError: LoadError: assertion failed: |fx - 1.0| <= 2.220446049250313e-12
  fx = 0.0
  1.0 = 1.0
  difference = 1.0 > 2.220446049250313e-12
while loading In[32], in expression starting on line 14

In [39]:
function newton(f, x; tol_f = 1e-6, tol_x = 1e-3)
    g(x) = ForwardDiff.derivative(f, x)
    d = -f(x)/g(x)
    iter = 0
    el_time = 0.0
    while abs(f(x)) > tol_f || abs(d) > tol_x
        x = x + d
        d = -f(x)/g(x)
    end
    return x, ef, f(x), g(x), iter, el_time
end

newton (generic function with 1 method)

In [42]:
newton(x->atan(x*sin(x)), 1)

(0.0005261967123839608,0,2.768829673463177e-7,0.0010523933276379003,0,0.0)

## Testes de funcionamento

Essencialmente duas coisas
- Não quebre
    - Domínio das funções (raiz de negativo, divisão por zero)
    - Teorias de convergência (específico do método)
    - Escala
- Não faça loop infinito
    - Máximo de iterações, avaliações de função, tempo, etc.
    - Condições passíveis de falha

In [15]:
context("Não quebre") do
    facts("f'(x) = 0") do
        f(x) = x^2 - 1
        x0 = 0.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact x --> 0.0
        @fact ef --> 0
        @fact fx --> 0.0
        @fact fdx --> 0.0
        @fact iter --> less_than(1000)
        @fact el_time --> less_than(60)
    end
    facts("Escala") do
        for p = [1e8; 1e-8]
            f(x) = p * (x^2 - 4)
            x, ef, fx, fdx, iter, el_time = newton(f, x0)
            @fact x --> 2.0
            @fact ef --> 0
        end
    end
    facts("Longe da solução") do
        f(x) = x^2/(1+x^2)
        x0 = 2.0
        x, ef, fx, fdx, iter, el_time = newton(f, x0)
        @fact abs(x) --> greater_than(1e6)
        @fact ef --> not(0)
    end
end

context("Sem loop infinito") do
    f(x) = x^4
    x0 = 1.0
    facts("Iterações") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxiter=1)
        @fact ef --> not(0)
        @fact iter --> 1
    end
    
    facts("Tempo") do
        x, ef, fx, fdx, iter, el_time = newton(f, x0, maxtime=0.0)
        @fact ef --> not(0)
    end
end

  > Não quebre
f'(x) = 0
6 facts verified.
Escala
    Failure :: (line:-1) :: Não quebre :: fact was false
      Expression: x --> 2.0
        Expected: 2.0
        Occurred: 0.0
    Failure :: (line:-1) :: Não quebre :: fact was false
      Expression: x --> 2.0
        Expected: 2.0
        Occurred: 0.0
Out of 4 total facts:
  Verified: 2
  Failed:   2
Longe da solução
    Failure :: (line:-1) :: Não quebre :: fact was false
      Expression: abs(x) --> greater_than(1.0e6)
        Expected: 0.0 > 1.0e6
    Failure :: (line:-1) :: Não quebre :: fact was false
      Expression: ef --> not(0)
        Expected: 0 --> not(0)
Out of 2 total facts:
  Failed:   2
  > Sem loop infinito
Iterações


LoadError: LoadError: ArgumentError: function newton does not accept keyword arguments
while loading In[15], in expression starting on line 30